# Preprocessing

In [1]:
!pip install hazm

     |████████████████████████████████| 317kB 12.7MB/s 
     |████████████████████████████████| 235kB 17.3MB/s 
     |████████████████████████████████| 1.4MB 26.6MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153929 sha256=b3014767336ac1e69eddfbc63fabe4fcf2ce8ddc5beef94ad29a61f8c5dcad2d
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394486 sha256=55c3e638fdc21f0f246f3843e7aebe0a1f2a5ccd89d0221d5ad51ad0b361558b
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
!pip install torchinfo

In [2]:
import numpy as np
import scipy as sc
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
import random
import re
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils import clip_grad_norm_
from __future__ import unicode_literals
from hazm import *
import json
from torch.autograd import Variable
import torch.nn.functional as F

In [3]:
!apt install unrar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.5.8-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [4]:
random.seed(1024)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unrar e /content/drive/MyDrive/projects/language_model/News.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/projects/language_model/News.rar

Extracting  train.csv                                                      5% 11% 16% 22% 27% 33% 38% 44% 49% 55% 61% 66% 72% 77% 83% 84%  OK 
Extracting  test.csv                                                      89% 95% 99%  OK 
All OK


In [21]:
train_data = pd.read_csv("./train.csv",sep="\t", error_bad_lines= False , encoding= 'utf-8')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
ex = list(range(238, 264))
ex.remove(243)
excludes = list(range(45,52)) + list(range(53,58)) + list(range(59,73)) + list(range(74,87)) + list(range(88,97)) + list(range(98,102)) + [104, 136, 137, 142, 209 , 210, 214, 215, 216, 224, 225, 235] + list(range(218, 222)) + ex
class Preprocessor:

  def __init__(self, df, excludes=excludes):
    self.train_data = df
    self.train_data = self.train_data.dropna(subset=['category'])
    self.train_data = self.train_data.loc[self.train_data['category']!='category']
    self.excludes = excludes
    self.clean()
    self.count_chars()
    self.tokens = self.tokenize()

  def clean(self):
    replaced = []
    for sentence in self.train_data.text.astype(str):
      replaced.append(re.sub("\d+","N",sentence))
    ws = []
    # normalizer = Normalizer()
    for a in replaced:
      # st = normalizer.normalize(a)
      w = ["\s"] + [tk for tk in word_tokenize(a) if tk.isalpha()] + ['\e']
      ws.append(' '.join(w))
    self.cleaned_list = ws

  def count_chars(self):
    word2index = {}
    index2word = {}
    total_words= sum([len(token) for sent in self.cleaned_list for token in sent])
    for sent in self.cleaned_list:
      for char in sorted(list(set(sent))):
        if word2index.get(char) is None:
            word2index[char] = len(word2index)
    index2word = {v:k for k,v in word2index.items()}
    uniques = len(index2word)
    print(f"\nTotal characters = {total_words}")
    print(f"\nUnique characters = {uniques}")
    with open("w2i.txt","w") as file:
      file.write(json.dumps(word2index,indent=4,ensure_ascii=False))
    with open("i2w.txt","w") as file:
      file.write(json.dumps(index2word,indent=4,ensure_ascii=False))
    self.word2index = word2index
    self.index2word = index2word


  def tokenize(self):
    tokens = []
    for sent in self.cleaned_list:
      for char in list(sent):
        if(self.word2index[char] not in self.excludes):
          tokens.append(self.word2index[char])
    data = torch.tensor(tokens).to(device)
    data = torch.unsqueeze(data, dim=1) 
    return data

In [22]:
pre = Preprocessor(train_data)


Total characters = 146940000

Unique characters = 277


In [ ]:
pre.cleaned_list[:5]

['\\s به گزارش خبرنگار حوزه میراث و فرهنگی باشگاه خبرنگاران جوان محمد حسن خان اعتماد السلطنه به نقل از برخی منابع اسفراین چمن کالپوش چمن کالپوش بر اساس آنچه در اعتقادات اهالی منطقه مشهود است آخرین منزلگاه داریوش سوم است بنابراین باید دشت اسفراین را جولانگاه مقدونیان دانست این دشت از دستبرد و سم ستوران یونانی در امان بنا بر این گزارش شهر اسفراین امروزی در شمال غربی استان خراسان شمالی قرار این شهر دربرگیرنده بیش از N بقعه از بزرگان و امامزادگان است شایان ذکر است شهر فعلی اسفراین از N محله تشکیل شده و زیستگاه حیواناتی چون آهو گرگ گورکن خرگوش روباه گراز پلنگ کفتار و بز کوهی است منطقه ساری گل در شمال شرقی اسفراین قرار گفتنی است ابوعبدلله حمد بن احمد مقدسی در خصوص اسفراین اسفراین روستایی بزرگ و مرکز انگور خوب و کشتزار است جاده گرگان آن را به دو نیمه شهر آن به همین نام است آباد و گرانقدر است از نهری که از کوه در این روستا از آن نیست مردم آن اهل حدیثند خاطرنشان ابوعبدالله الحاکم مؤلف تاریخ نیشابور نیز اسفراین را شهری خوش آب و هوا ذکر است و قباد پادشاه ساسانی نام اسفراین را به علت داشتن آب و هو

In [ ]:
pre.tokens.shape

torch.Size([145323109, 1])

In [ ]:
len(excludes)

93

## Model section

---



In [11]:
class LanguageModel(nn.Module):
  """
  """

  def __init__(self, input_size, num_layers, output_size, hidden_size, dropout=0.25, bidirectional=False):
    super(LanguageModel, self).__init__()
    self.input_size = input_size
    self.num_layers = num_layers
    self.output_size = output_size
    self.hidden_size = hidden_size
    self.dropout = dropout
    self.bidirectional = bidirectional
    self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, dropout=self.dropout, bidirectional=self.bidirectional, batch_first=True)
    self.linear = nn.Linear(self.hidden_size, self.output_size)
  
  def forward(self, input, hidden):
    output, _hidden = self.lstm(input, hidden)
    linear = self.linear(output.contiguous().view(output.size(0) * output.size(1), -1))
    # print(linear.shape,"*********")
    return linear, _hidden 

In [12]:
model = LanguageModel(1,2,277,512).to(device)
# model = LanguageModel(1,2,277,512).to(device)


In [14]:
NUM_EPOCHS = 1
BATCH_SIZE = 256
SEQ_LENGTH = 30
HIDDEN_SIZE = 512
NUM_LAYERS = 2
learning_rate = 0.002

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

3298069

In [ ]:
torch.unsqueeze(pre.tokens[0:10],-1).shape

torch.Size([10, 1, 1])

In [ ]:
x = torch.reshape(torch.unsqueeze(pre.tokens[0:10],-1), (10,1,1))

In [ ]:
y = (torch.zeros((4,1,512)).to(device), torch.zeros((4,1,512)).to("cpu"))

In [ ]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.shape[0] // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = torch.narrow(data, 0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).contiguous()
    # if USE_CUDA:
    #     data = data.cuda()
    return data


def getBatch(data, seq_length):
     for i in range(0, data.shape[1] - seq_length, seq_length):
        inputs = Variable(torch.unsqueeze(data[:, i: i + seq_length], 2))
        targets = Variable(data[:, (i + 1): (i + 1) + seq_length].contiguous())
        # targets = torch.unsqueeze(targets, 2)
        yield (inputs, targets)

In [ ]:
train_data = batchify(pre.tokens, BATCH_SIZE)

In [ ]:
train_data.shape

torch.Size([256, 573984])

In [ ]:
pre.tokens.shape

torch.Size([146940000, 1])

In [ ]:
277-93

184

In [ ]:
import os
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# if os.path.exists("checkpoints.cp"):
#   print("The pretrained model has been found")
#   checkpoint = torch.load("checkpoints.cp")
#   model.load_state_dict(checkpoint['model_state_dict'])
#   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#   epoch = checkpoint['epoch']
#   loss = checkpoint['loss']

# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states] 

# Train the model
train_loss = []
plxs = []
for epoch in range(NUM_EPOCHS):
    # Set initial hidden and cell states
    # states = (torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE),
    #           torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE))
    states = (torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).float().to(device),
              torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).float().to(device))
    losses = []
    plx = []
    for i,batch in enumerate(getBatch(train_data, SEQ_LENGTH)):
        # Get mini-batch inputs and targets
        inputs, targets = batch
        # Forward pass
        # print(i,inputs.shape, targets.shape)
        states = detach(states)
        outputs, states = model(inputs.float(), states)
        # print(targets, targets.shape)
        loss = criterion(outputs, targets.reshape(-1))
        losses.append(loss.item())
        plx.append(np.exp(loss.item()))
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1)
        # step = (i+1) // SEQ_LENGTH
        num_batches = train_data.shape[1] // SEQ_LENGTH
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, NUM_EPOCHS, step, num_batches, loss.item(), np.exp(loss.item())))
        # break
    ls = np.mean(losses)
    ps = np.mean(plx)
    train_loss.append(ls)
    plxs.append(ps)
    print(f"Epoch={epoch}***********************Loss={ls}*****************Perplexity={ps}")
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "checkpoints.cp")


Epoch [1/1], Step[100/18922], Loss: 2.7438, Perplexity: 15.55
Epoch [1/1], Step[200/18922], Loss: 2.6310, Perplexity: 13.89
Epoch [1/1], Step[300/18922], Loss: 2.5544, Perplexity: 12.86
Epoch [1/1], Step[400/18922], Loss: 2.4468, Perplexity: 11.55
Epoch [1/1], Step[500/18922], Loss: 2.4000, Perplexity: 11.02
Epoch [1/1], Step[600/18922], Loss: 2.3512, Perplexity: 10.50
Epoch [1/1], Step[700/18922], Loss: 2.3302, Perplexity: 10.28
Epoch [1/1], Step[800/18922], Loss: 2.2693, Perplexity:  9.67
Epoch [1/1], Step[900/18922], Loss: 2.2168, Perplexity:  9.18
Epoch [1/1], Step[1000/18922], Loss: 2.1969, Perplexity:  9.00
Epoch [1/1], Step[1100/18922], Loss: 2.1783, Perplexity:  8.83
Epoch [1/1], Step[1200/18922], Loss: 2.1055, Perplexity:  8.21
Epoch [1/1], Step[1300/18922], Loss: 2.1237, Perplexity:  8.36
Epoch [1/1], Step[1400/18922], Loss: 2.0427, Perplexity:  7.71
Epoch [1/1], Step[1500/18922], Loss: 2.0450, Perplexity:  7.73
Epoch [1/1], Step[1600/18922], Loss: 2.0393, Perplexity:  7.69
E

KeyboardInterrupt: ignored

In [ ]:
import datetime
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f"checkpoints_{str(int(datetime.datetime.now().timestamp()))}.cp")

In [ ]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  with torch.no_grad():
    state = (torch.zeros(NUM_LAYERS, 1, HIDDEN_SIZE).float().to(device),
              torch.zeros(NUM_LAYERS, 1, HIDDEN_SIZE).float().to(device))
    word = prime_str

    idx = pre.word2index[prime_str[-1]]
    input = Variable(torch.Tensor([idx])).float().unsqueeze(1).to(device)
    for i in range(predict_len):
        # Forward propagate RNN 
        # print(input.shape)
        output, state = model(input.unsqueeze(0), state)

        # Sample a word id
        prob = output.exp()
        word_id = torch.multinomial(prob, num_samples=1).item()

        # Fill input with sampled word id for the next time step
        input.fill_(word_id)

        # File write
        word = word + pre.index2word[word_id]
        # word = '\n' if word == '<eos>' else word + ' '
        # f.write(word)

    print(word)
    return word

In [ ]:
txt = "سینا عباسی"
evaluate(txt)

سینا عباسینرته نا غیکروا بمگات به خوبر میرت خابنا ترگ غد وي ازمامت وادث خضن کادر يمای مايتهی در سسر بو خرا دس 


'سینا عباسینرته نا غیکروا بمگات به خوبر میرت خابنا ترگ غد وي ازمامت وادث خضن کادر يمای مايتهی در سسر بو خرا دس '

### Test results

### **GRU**

In [ ]:
class GRULanguageModel(nn.Module):
  """
  """

  def __init__(self, input_size, num_layers, output_size, hidden_size, dropout=0.25, bidirectional=False):
    super(GRULanguageModel, self).__init__()
    self.input_size = input_size
    self.num_layers = num_layers
    self.output_size = output_size
    self.hidden_size = hidden_size
    self.dropout = dropout
    self.bidirectional = bidirectional
    self.lstm = nn.GRU(self.input_size, self.hidden_size, self.num_layers, dropout=self.dropout, bidirectional=self.bidirectional, batch_first=True)
    self.linear = nn.Linear(self.hidden_size, self.output_size)
  
  def forward(self, input, hidden):
    output, _hidden = self.lstm(input, hidden)
    linear = self.linear(output.contiguous().view(output.size(0) * output.size(1), -1))
    # print(linear.shape,"*********")
    return linear, _hidden

In [ ]:
gru = GRULanguageModel(1,2,277,512).to(device)

In [ ]:
train_data = batchify(pre.tokens, BATCH_SIZE)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)

# if os.path.exists("checkpoints.cp"):
#   print("The pretrained model has been found")
#   checkpoint = torch.load("checkpoints.cp")
#   model.load_state_dict(checkpoint['model_state_dict'])
#   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#   epoch = checkpoint['epoch']
#   loss = checkpoint['loss']

# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states] 

# Train the model
train_loss = []
plxs = []
for epoch in range(10):
    # Set initial hidden and cell states
    # states = (torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE),
    #           torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE))
    states = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).float().to(device)
    losses = []
    plx = []
    for i,batch in enumerate(getBatch(train_data, SEQ_LENGTH)):
        # Get mini-batch inputs and targets
        inputs, targets = batch
        # Forward pass
        # print(i,inputs.shape, targets.shape)
        states = states.detach()
        outputs, states = gru(inputs.float(), states)
        # print(targets, targets.shape)
        loss = criterion(outputs, targets.reshape(-1))
        losses.append(loss.item())
        plx.append(np.exp(loss.item()))
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(gru.parameters(), 0.5)
        optimizer.step()

        step = (i+1)
        # step = (i+1) // SEQ_LENGTH
        num_batches = train_data.shape[1] // SEQ_LENGTH
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, NUM_EPOCHS, step, num_batches, loss.item(), np.exp(loss.item())))
        # if step%20:
        #   break
        # break
    ls = np.mean(losses)
    ps = np.mean(plx)
    train_loss.append(ls)
    plxs.append(ps)
    print(f"Epoch={epoch}***********************Loss={ls}*****************Perplexity={ps}")
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "checkpoints.cp")


Epoch [1/1], Step[100/18922], Loss: 2.5790, Perplexity: 13.18
Epoch [1/1], Step[200/18922], Loss: 2.4012, Perplexity: 11.04
Epoch [1/1], Step[300/18922], Loss: 2.3440, Perplexity: 10.42
Epoch [1/1], Step[400/18922], Loss: 2.2185, Perplexity:  9.19
Epoch [1/1], Step[500/18922], Loss: 2.2200, Perplexity:  9.21
Epoch [1/1], Step[600/18922], Loss: 2.1504, Perplexity:  8.59
Epoch [1/1], Step[700/18922], Loss: 2.1539, Perplexity:  8.62
Epoch [1/1], Step[800/18922], Loss: 2.1046, Perplexity:  8.20
Epoch [1/1], Step[900/18922], Loss: 2.0524, Perplexity:  7.79
Epoch [1/1], Step[1000/18922], Loss: 2.0647, Perplexity:  7.88
Epoch [1/1], Step[1100/18922], Loss: 2.0614, Perplexity:  7.86
Epoch [1/1], Step[1200/18922], Loss: 1.9925, Perplexity:  7.33
Epoch [1/1], Step[1300/18922], Loss: 2.0226, Perplexity:  7.56
Epoch [1/1], Step[1400/18922], Loss: 1.9347, Perplexity:  6.92
Epoch [1/1], Step[1500/18922], Loss: 1.9545, Perplexity:  7.06
Epoch [1/1], Step[1600/18922], Loss: 1.9640, Perplexity:  7.13
E

KeyboardInterrupt: ignored

In [ ]:
import datetime
torch.save({
                    'epoch': epoch,
                    'model_state_dict': gru.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, f"./checkpoints_{str(int(datetime.datetime.now().timestamp()))}.cp")

In [ ]:
def gru_evaluate(prime_str='A', predict_len=100, temperature=0.8):
  with torch.no_grad():
    state = torch.zeros(NUM_LAYERS, 1, HIDDEN_SIZE).float().to(device)
    word = prime_str

    idx = pre.word2index[prime_str[-1]]
    input = Variable(torch.Tensor([idx])).float().unsqueeze(1).to(device)
    for i in range(predict_len):
        # Forward propagate RNN 
        # print(input.shape)
        output, state = gru(input.unsqueeze(0), state)

        # Sample a word id
        prob = output.exp()
        word_id = torch.multinomial(prob, num_samples=1).item()

        # Fill input with sampled word id for the next time step
        input.fill_(word_id)

        # File write
        word = word + pre.index2word[word_id]
        # word = '\n' if word == '<eos>' else word + ' '
        # f.write(word)

    print(word)
    return word

In [ ]:
txt = "سینا عباسی"
gru_evaluate(txt)

سینا عباسید باراني خرستهه ذلی ئو ورودان مشامف N به میرمی تنس دطووی فیست ماحاخهان است كهو افزان از N بیرج فهه م


'سینا عباسید باراني خرستهه ذلی ئو ورودان مشامف N به میرمی تنس دطووی فیست ماحاخهان است كهو افزان از N بیرج فهه م'

In [99]:
import datetime

class LanguageModelUtility:

  def __init__(self, model=None, w2i=None, i2w=None, hyperparameters={}):
    self.hyperparameters = hyperparameters
    NUM_EPOCHS = hyperparameters.get("NUM_EPOCHS", 1)
    self.BATCH_SIZE = hyperparameters.get("BATCH_SIZE", 1) 
    SEQ_LENGTH = hyperparameters.get("SEQ_LENGTH", 30)
    self.HIDDEN_SIZE = hyperparameters.get("HIDDEN_SIZE", 512) 
    self.NUM_LAYERS = hyperparameters.get("NUM_LAYERS", 2) 
    learning_rate = hyperparameters.get("learning_rate", 0.002)
    self.word2index = w2i
    self.index2word = i2w
    self.model = model
    

  @staticmethod
  def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.shape[0] // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = torch.narrow(data, 0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).contiguous()
    # if USE_CUDA:
    #     data = data.cuda()
    return data

  @staticmethod
  def getBatch(data, seq_length):
      for i in range(0, data.shape[1] - seq_length, seq_length):
          inputs = Variable(torch.unsqueeze(data[:, i: i + seq_length], 2))
          targets = Variable(data[:, (i + 1): (i + 1) + seq_length].contiguous())
          # targets = torch.unsqueeze(targets, 2)
          yield (inputs, targets)

  def define_model(self, model_type="lstm"):
    if model_type=="lstm":
      return LanguageModel(1,2,277,512).to(device)
    else:
      return GRULanguageModel(1,2,277,512).to(device)

  def train_char_lstm(self, model, hyperparameters, data):
    self.hyperparameters = hyperparameters
    NUM_EPOCHS = hyperparameters.get("NUM_EPOCHS", 1)
    self.BATCH_SIZE = hyperparameters.get("BATCH_SIZE", 256) 
    SEQ_LENGTH = hyperparameters.get("SEQ_LENGTH", 30)
    self.HIDDEN_SIZE = hyperparameters.get("HIDDEN_SIZE", 512) 
    self.NUM_LAYERS = hyperparameters.get("NUM_LAYERS", 2) 
    learning_rate = hyperparameters.get("learning_rate", 0.002)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    def detach(states):
        return [state.detach() for state in states] 

    train_data = LanguageModelUtility.batchify(data, BATCH_SIZE)
    # Train the model
    train_loss = []
    plxs = []
    for epoch in range(NUM_EPOCHS):
        # Set initial hidden and cell states
        # states = (torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE),
        #           torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE))
        states = (torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).float().to(device),
                  torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).float().to(device))
        losses = []
        plx = []
        for i,batch in enumerate(LanguageModelUtility.getBatch(train_data, SEQ_LENGTH)):
            # Get mini-batch inputs and targets
            inputs, targets = batch
            # Forward pass
            # print(i,inputs.shape, targets.shape)
            states = detach(states)
            outputs, states = model(inputs.float(), states)
            # print(targets, targets.shape)
            loss = criterion(outputs, targets.reshape(-1))
            losses.append(loss.item())
            plx.append(np.exp(loss.item()))
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()

            step = (i+1)
            # step = (i+1) // SEQ_LENGTH
            num_batches = train_data.shape[1] // SEQ_LENGTH
            if step % 100 == 0:
                print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                      .format(epoch+1, NUM_EPOCHS, step, num_batches, loss.item(), np.exp(loss.item())))
            # break
        ls = np.mean(losses)
        ps = np.mean(plx)
        train_loss.append(ls)
        plxs.append(ps)
        print(f"Epoch={epoch}***********************Loss={ls}*****************Perplexity={ps}")
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, f"./checkpoints_{str(int(datetime.datetime.now().timestamp()))}.cp")
  

  def get_next_states_and_output(self, hidden, _input="ب"):
    with torch.no_grad():
      def detach(states):
        return [state.detach() for state in states]
      
      idx = []

      for chr in _input:
        idx.append(self.word2index[chr])
      input = Variable(torch.Tensor(idx)).float().unsqueeze(1).to(device)
      output, state = self.model(input.unsqueeze(0), hidden)
      return output, state
  
  def convert_prefix_to_hiddens(self, init_hiddens, prefix):
    list_of_hiddens = [init_hiddens]
    
    hidden = init_hiddens
    output = None
    for chr in prefix:
      output , hidden = self.get_next_states_and_output(hidden, chr)
      list_of_hiddens.append(hidden)
    return list_of_hiddens, output

  def get_probes(self, init_hiddens, prefix):
    _, output = self.convert_prefix_to_hiddens(init_hiddens, prefix)
    return F.softmax(output[-1].view(1,-1))
  
  def get_next_char(self, prefix):

    states = (torch.zeros(self.NUM_LAYERS, self.BATCH_SIZE, self.HIDDEN_SIZE).float().to(device),
              torch.zeros(self.NUM_LAYERS, self.BATCH_SIZE, self.HIDDEN_SIZE).float().to(device))
    output, _ = self.get_next_states_and_output(states, prefix)
    index = torch.argmax(output[-1]).item()
    return self.index2word[index], torch.max(F.softmax(output)).item()

  def generate_text(self, prefix, num_sample):
    sent = prefix
    for _ in range(num_sample):
      if sent[-2:]=="\e":
        break
      sent = sent + self.get_next_char(sent)[0]
    return sent
  
  def get_overall_prob(self, sentance):
    res = 0
    states = (torch.zeros(self.NUM_LAYERS, self.BATCH_SIZE, self.HIDDEN_SIZE).float().to(device),
              torch.zeros(self.NUM_LAYERS, self.BATCH_SIZE, self.HIDDEN_SIZE).float().to(device))
    for ptr in range(len(sentance)-1):
      prefix = sentance[ptr]
      chr = sentance[ptr+1]
      prob = self.get_probes(states, prefix)
      res += np.log(prob[0][self.word2index[chr]].item())
    return res
  
  def evaluate(self, prefix, target):
    # predicted = self.generate_text(prefix, 500)
    criterion = nn.CrossEntropyLoss()
    pp = 0
    with torch.no_grad():
      states = (torch.zeros(NUM_LAYERS, 1, HIDDEN_SIZE).float().to(device),
                  torch.zeros(NUM_LAYERS, 1, HIDDEN_SIZE).float().to(device))

      idx = [pre.word2index[x] for x in prefix]
      input = Variable(torch.Tensor(idx)).float().unsqueeze(1).to(device)
      # print(len(input))
      # import pdb;pdb.set_trace()
      target_id = [pre.word2index.get(target[j], 0) for j in range(1,len(input)+1)]
      target_tsn = Variable(torch.Tensor(target_id)).long().unsqueeze(1).to(device)
      for i in range(len(target)-len(prefix)):
          # Forward propagate RNN 
          # print(input.shape)
          output, states = self.model(input.unsqueeze(0), states)
          
          # Sample a word id
          prob = output.exp()
          # print(input.unsqueeze(0), input.unsqueeze(0).shape)
          # print(target_tsn, target_tsn.shape)
          # print(output, output.shape)
          loss = criterion(output, target_tsn.reshape(-1))
          target_id = target_id[1:] + [pre.word2index[target[len(prefix) + 1]]]
          target_tsn = Variable(torch.Tensor(target_id)).long().unsqueeze(1).to(device)
          pp+=np.exp(loss.item())
          # print(prob.shape)
          word_id = torch.multinomial(prob[-1], num_samples=1).item()
          # print(prob.shape, word_id)
          input = Variable(torch.Tensor(input.squeeze().tolist()[1:] + [word_id])).float().unsqueeze(1).to(device)
          # Fill input with sampled word id for the next time step
          # input.f

          # word = word + pre.index2word[word_id]
    
      return pp/(len(target)-len(prefix))




      



In [100]:
import os
states = (torch.zeros(NUM_LAYERS, 1, HIDDEN_SIZE).float().to(device),
                  torch.zeros(NUM_LAYERS, 1, HIDDEN_SIZE).float().to(device))
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# if os.path.exists("./drive/MyDrive/checkpoints_1624289195.cp"):
#   print("The pretrained model has been found")
#   checkpoint = torch.load("./drive/MyDrive/checkpoints_1624289195.cp")
#   model.load_state_dict(checkpoint['model_state_dict'])
#   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#   epoch = checkpoint['epoch']
#   loss = checkpoint['loss']
util = LanguageModelUtility(model,pre.word2index, pre.index2word)



In [ ]:
_input = "ف"
output , next_state = util.get_next_states_and_output(states, _input)
print("Output=", output)
print("Output shape=", output.shape)

print("hidden_state=", next_state[0], "Cell state=", next_state[1])
print("hidden(cell) state shape=", next_state[0].shape)



Output= tensor([[ 4.6075,  0.1992, -4.5678, -5.3531, -5.0582, -2.7137, -2.3740,  2.7629,
          1.0050,  3.3218, -0.8912,  0.8100, -0.6034, -1.0316,  1.3652,  0.3856,
          2.0447,  0.8974,  2.3673,  0.5585, -1.9625, -0.2292, -0.4599, -1.1966,
          1.8510, -0.5564,  1.3664,  3.0950,  3.2626,  3.3074,  1.7969,  0.5155,
          0.5588,  2.2963,  0.3217,  2.7163, -0.2603, -2.2243, -1.1806, -0.9910,
         -3.4525, -2.1922, -1.9341,  1.8414,  2.4048, -6.3459, -6.4614, -6.7595,
         -5.9583, -6.7817, -6.4169, -6.9276, -0.8000, -6.6361, -6.6370, -6.5746,
         -7.0253, -6.4972,  1.5187, -6.9483, -6.8818, -6.3963, -6.6156, -6.9895,
         -6.6762, -6.5673, -7.0769, -6.9202, -6.4102, -6.6176, -6.5587, -7.0669,
         -6.2581, -4.3206, -6.5673, -6.1777, -6.7198, -6.5960, -6.1278, -6.5445,
         -6.5896, -6.4745, -6.6243, -6.6407, -7.1152, -7.1129, -7.2281, -3.2284,
         -6.8362, -6.6189, -5.9691, -6.5603, -6.8416, -6.8326, -6.6945, -6.8182,
         -6.6819, -5

In [ ]:
_input = "فا"
output , next_state = util.get_next_states_and_output(states, _input)
print("Output=", output)
print("Output shape=", output.shape)

print("hidden_state=", next_state[0], "Cell state=", next_state[1])
print("hidden(cell) state shape=", next_state[0].shape)

Output= tensor([[  4.5170,   0.3484,  -4.6409,  -5.6567,  -5.1305,  -2.8607,  -2.1282,
           2.8689,   1.7849,   3.3922,  -0.6367,   1.2038,  -0.3218,  -0.8052,
           2.1555,   0.5042,   1.6683,   1.5742,   1.6980,   0.3958,  -1.6498,
          -0.6050,  -0.0295,  -0.5895,   1.9981,   0.0533,   1.4768,   2.8811,
           2.9274,   3.8882,   1.7446,   1.1503,   0.1581,   2.1992,   0.8507,
           2.4801,   0.2106,  -1.9189,  -1.8717,  -0.9983,  -3.3818,  -2.2375,
          -2.1188,   1.6977,   2.1969,  -6.5221,  -6.5589,  -6.8761,  -6.0544,
          -6.8995,  -6.5690,  -7.0897,  -1.0445,  -6.7279,  -6.7388,  -6.6783,
          -7.1426,  -6.5217,   1.4406,  -7.0531,  -7.0495,  -6.5243,  -6.7443,
          -7.0899,  -6.7888,  -6.6538,  -7.1671,  -7.0122,  -6.5364,  -6.7313,
          -6.6384,  -7.1408,  -6.3611,  -4.4432,  -6.7170,  -6.2917,  -6.8026,
          -6.7487,  -6.2280,  -6.6931,  -6.7666,  -6.5941,  -6.7713,  -6.7370,
          -7.1950,  -7.2100,  -7.3709,  -3.1

In [ ]:
_input = "فا"
hiddens , outout = util.convert_prefix_to_hiddens(states, _input)
print("hidden=", hiddens)
print("hiddens length=", len(hiddens))
print("hidden[1]=", hiddens[1])

print("output=",output)

hidden= [(tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]]), tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])), (tensor([[[-2.0199e-02, -1.8881e-03, -4.7276e-03,  ..., -8.3166e-04,
          -2.0093e-01, -9.5539e-02]],

        [[-2.4794e-01, -1.6078e-03, -1.2269e-07,  ...,  1.4866e-01,
           9.7701e-15, -3.7643e-05]]]), tensor([[[-2.9303e-01, -3.0458e-01, -2.2485e-01,  ..., -4.3065e-01,
          -3.6108e-01, -4.4730e-01]],

        [[-8.2873e-01, -1.3999e-02, -2.8136e-04,  ...,  1.5303e-01,
           3.4087e-08, -1.1355e-03]]])), (tensor([[[-5.0409e-03, -7.2002e-01, -6.6328e-01,  ..., -4.8946e-01,
          -7.7167e-01, -1.1736e-01]],

        [[-7.7945e-02, -6.6050e-08, -1.5352e-15,  ..., -6.6687e-01,
           7.3510e-13, -1.7651e-14]]]), tensor([[[-1.1059e+00, -1.1334e+00, -1.0267e+00,  ..., -8.9240e-01,
          -1.1121e+00, -1.2038e+00]],

        [[-1.8188e+00, -6.0880e-04, -1.1080e-08,  ..., -8.06

In [ ]:
_input = "فا"
outout = util.get_probes(states, _input)
print(f"Input=========={_input}")
print("probes=", outout)
print("Sum of probes are=",outout.sum().item())


_input = "من"
outout = util.get_probes(states, _input)
print(f"Input=========={_input}")
print("probes=", outout)
print("Sum of probes are=",outout.sum().item())

_input = "شهر"
outout = util.get_probes(states, _input)
print(f"Input=========={_input}")
print("probes=", outout)
print("Sum of probes are=",outout.sum().item())



Input==========فا
probes= tensor([[9.1109e-02, 9.7424e-05, 2.5147e-08, 9.8989e-09, 2.3674e-08, 3.2797e-04,
         2.2212e-04, 1.6881e-01, 1.4430e-02, 1.1877e-01, 3.7593e-05, 1.7546e-03,
         9.1116e-04, 2.7495e-03, 5.1654e-02, 1.2330e-04, 9.4157e-03, 2.0700e-03,
         5.4984e-03, 5.1233e-03, 7.0256e-05, 3.5753e-04, 8.2234e-04, 5.4287e-04,
         5.0699e-03, 5.3120e-04, 2.1894e-03, 3.2470e-02, 1.0267e-01, 5.4284e-03,
         2.4165e-01, 4.1696e-03, 1.6926e-03, 1.9156e-02, 5.1333e-03, 7.6015e-02,
         1.4346e-04, 2.0183e-04, 6.5174e-04, 1.2997e-03, 1.6920e-05, 1.0155e-05,
         3.8765e-05, 6.8507e-03, 1.4694e-02, 1.1729e-08, 1.8839e-08, 1.0477e-08,
         2.1692e-08, 8.0400e-09, 1.0831e-08, 9.5098e-09, 8.0102e-06, 1.1105e-08,
         1.0344e-08, 1.4758e-08, 7.1342e-09, 1.3939e-08, 5.0024e-03, 6.3114e-09,
         1.0244e-08, 1.2359e-08, 1.3839e-08, 6.0949e-09, 1.3354e-08, 1.1834e-08,
         7.9888e-09, 9.3583e-09, 9.4502e-09, 1.1412e-08, 1.1753e-08, 7.3832e-09,
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:132: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
_input = "انتخا"
prob , outout = util.get_next_char(_input)
print(f"Input=========={_input}")
print("probe=",prob,'\n', "Char=", outout )

Input==========انتخا
probe= ن 
 Char= 0.8138872981071472


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
_input = "انتخا"
output = util.generate_text(_input, 100)
print(f"Input=========={_input}")
print("probe=",prob,'\n', "Char=", output )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Input==========انتخا
probe= ن 
 Char= انتخان به این مورد است که در این روز است که در این مورد است و از این موانع است و این موانع است و این مورد


In [ ]:
_input = "یاد"
output = util.generate_text(_input, 1000)
print(f"Input=========={_input}")
print("probe=",prob,'\n', "Char=", output )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Input==========یاد
probe= ن 
 Char= یاد از این مواطق است که در این مواده این مواد می شود و به این مواد می شود از این مورد این مواد می شود و این مواد می شود و به این مواده از این بازی از این است که در این موانع این مورد است و این مواد می شود و از این مواد می شود و به این مواد می شود و از این مواد می شود و در این مواطق است که در این موضوع است که از این مورد است که در این موار می شود و در این موانقات است که در این موانع است که از این موضوع است که این موان است که در این موار می شود و این مواده از این موار می شود و این مواد می شود و در این مرابقه است اما این مواد دارد و است که در این مواد می شود و این مواد می شود و به این مورد این مواده است و از این مواد می شود این موانع است که این مورد است که در این مواد می شود این مورد این مواد داد و از این مواد می شود و این مورد این موران است که در این مواد می شود و از این مواد کرد و این مرابقه است که در این مواد می شود و از این مواده از این مواد مراور است که این مواد است و این مواد می شود و این مواطق است و از این مواد مراوره است که در این مورد است و این

In [ ]:
_input = "مقد"
output = util.generate_text(_input, 200)
print(f"Input=========={_input}")
print("probe=",prob,'\n', "Char=", output )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Input==========مقد
probe= ن 
 Char= مقد از این مواد می شود و این مواد می شود و این امتخابات است و در این مورد این مواد می شود و این مورد است که این مورد این مورد این مواده از این مواده است و از این مواد می شود و این موانقت و به این مواده ا


In [ ]:
_input = "من محمد مقدس هستم"
output = util.get_overall_prob(_input)
print(f"Input=========={_input}")
print("probe=",output)

Input==========من محمد مقدس هستم
probe= -58.14199971017963


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:132: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
_input = "ییی یییی ییی ییییی"
output = util.get_overall_prob(_input)
print(f"Input=========={_input}")
print("probe=",output)

Input==========ییی یییی ییی ییییی
probe= -58.53320055751033


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:132: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [9]:
test_data = pd.read_csv("./test.csv",sep="\t", error_bad_lines= False , encoding= 'utf-8')

In [10]:
test_pre = Preprocessor(test_data)


Total characters = 26599835

Unique characters = 206


In [101]:
_input = test_pre.cleaned_list
for sample in _input:
  tokens = sample.split()
  output = util.evaluate(' '.join(tokens[:7]), sample)
  print(f"Test sample=========={sample}")
  print("probe=",output)

Test sample==========\s به گزارشحوزه ادبیات باشگاه خبرنگاران این کتاب شامل عکس های هوایی از مناظر ایران با شعرهایی از مولف که دل نوشته های وی روی تصاویر است به دو زبان فارسی و انگلیسی طراحی و به چاپ این کتاب در قطع خشتی و N صفحه تمام رنگی با تیراژ N عدد توسط نشر تحریر خیال منتشر شده و در اختیار علاقمندان قرار ویژگی عکس های هوایی این کتاب انتزاعی بودن تاکید بر فرم و جزئیات تصاویر می باشد که کمتر در عکس های هوایی عکاسان ایرانی یونس کلاهدوز استاد خلبانی است که زیادی را برای ثبت این لحظه ها تلاش نموده و در مقدمهکتاب چشم آسمان می نویسد این کتاب نگاهی است از آسمان به زمین و زندگی فارغ از قواعد معمول عکاسی با توجه به جزئیات تصاویر برخلاف آنچه در عکس های هوایی ایران رایج است این کتاب عکس نوشته هایی است در لحظه های منحصر بفرد و شوق انتقال این احساس و نگاه به مخاطبان می باشد لازم به ذکر است پیش از این عکاس سوئیسی به نام گئورگ گریستر با شباهتهایی به لحاظ سبک عکاسی ایران از آسمان به تصویر که نمایشگاهی را نیز با عنوان بهشت گمشده بر فراز ایران را به نمایش درآورد قیمت پشت جلد این کتاب N هزار تومان اس

KeyboardInterrupt: ignored